In [34]:
import pandas as pd
import pyarrow
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit,when, split
from pyspark.sql.types import IntegerType

In [2]:
spark = SparkSession.builder.appName('Ops').getOrCreate()

In [3]:
!curl https://raw.githubusercontent.com/oconnoj1/Lab_6_1/refs/heads/main/data/ReverseCodingItems.csv -o ./sample_data/ReverseCodingItems.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3157  100  3157    0     0  15704      0 --:--:-- --:--:-- --:--:-- 15785


In [4]:
!curl https://raw.githubusercontent.com/oconnoj1/Lab_6_1/refs/heads/main/data/health_survey.csv -o ./sample_data/health_survey.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  195k  100  195k    0     0   822k      0 --:--:-- --:--:-- --:--:--  819k


In [14]:
(reverse :=
 spark.read.csv('./sample_data/ReverseCodingItems.csv', header=True)
 .drop(col('Question'))
 .withColumnRenamed("Column Name", "Question")
)

DataFrame[Construct: string, Question # on Qualtrics Survey: string, Needs Reverse Coding?: string, Question: string]

In [12]:
(survey :=
 spark.read.csv('./sample_data/health_survey.csv', header=True)
)

DataFrame[ID: string, F1: string, F5: string, F2: string, F1_1: string, F2_1: string, F6: string, F4: string, F3: string, F5_1: string, F1_2: string, F2_2: string, F6_1: string, F2_3: string, F4_1: string, F2_4: string, F5_2: string, F2_5: string, F6_2: string, F1_3: string, F2_6: string, F5_3: string, F4_2: string, F2_7: string, F3_1: string, F2_8: string, F5_4: string, F3_2: string, F1_4: string, F3_3: string, F1_5: string, F5_5: string, F6_3: string, F1_6: string, F5_6: string, F2_9: string, F3_4: string, F4_3: string, F2_10: string, F1_7: string, F6_4: string, F4_4: string, F5_7: string, F3_5: string, F2_11: string]

In [15]:
(q_cols :=
 survey.drop(col('ID')).columns
)

['F1',
 'F5',
 'F2',
 'F1_1',
 'F2_1',
 'F6',
 'F4',
 'F3',
 'F5_1',
 'F1_2',
 'F2_2',
 'F6_1',
 'F2_3',
 'F4_1',
 'F2_4',
 'F5_2',
 'F2_5',
 'F6_2',
 'F1_3',
 'F2_6',
 'F5_3',
 'F4_2',
 'F2_7',
 'F3_1',
 'F2_8',
 'F5_4',
 'F3_2',
 'F1_4',
 'F3_3',
 'F1_5',
 'F5_5',
 'F6_3',
 'F1_6',
 'F5_6',
 'F2_9',
 'F3_4',
 'F4_3',
 'F2_10',
 'F1_7',
 'F6_4',
 'F4_4',
 'F5_7',
 'F3_5',
 'F2_11']

In [25]:
(reg := {'Strongly Agree':'5',
         'Somewhat Agree':'4',
         'Neither Agree nor Disagree':'3',
         'Somewhat Disagree':'2',
         'Strongly Disagree':'1',
 }
)

{'Strongly Agree': '5',
 'Somewhat Agree': '4',
 'Neither Agree nor Disagree': '3',
 'Somewhat Disagree': '2',
 'Strongly Disagree': '1'}

In [26]:
(rev := {'Strongly Agree':'1',
         'Somewhat Agree':'2',
         'Neither Agree nor Disagree':'3',
         'Somewhat Disagree':'4',
         'Strongly Disagree':'5',
 }
)

{'Strongly Agree': '1',
 'Somewhat Agree': '2',
 'Neither Agree nor Disagree': '3',
 'Somewhat Disagree': '4',
 'Strongly Disagree': '5'}

In [39]:

(
    survey_stacked :=
    survey
    .unpivot(ids = 'ID',
             values = q_cols,
             variableColumnName='Question',
             valueColumnName='Response')
    .join(reverse, on="Question", how="left")
    .withColumn("reg_coding", col('Response'))
    .withColumn("rev_coding", col('Response'))
    .replace(reg, subset=["reg_coding"])
    .replace(rev, subset=["rev_coding"])
    .withColumn("reg_coding", col('reg_coding').cast(IntegerType()))
    .withColumn("rev_coding", col('rev_coding').cast(IntegerType()))
    .withColumn("recoded_value", when(
        col('Needs Reverse Coding?') == 'Yes', col('rev_coding')
    ).otherwise(
        col('reg_coding')
    ))
    .withColumn('Question Type', split(col('Question'), '_').getItem(0))
    .groupBy('ID')
    .pivot('Question Type')
    .sum('recoded_value')
).toPandas()

,ID,F1,F2,F3,F4,F5,F6
0,125,32,47,23,18,29,18
1,124,37,56,22,17,37,19
2,7,36,56,22,19,29,22
3,51,34,52,24,17,29,23
4,169,35,49,20,20,27,17
...,...,...,...,...,...,...,...
259,50,32,49,22,19,29,16
260,57,39,55,21,24,32,18
261,45,38,58,17,21,32,20
262,216,32,49,17,18,28,16
